# Testing Individual components of the FV HE scheme

In [37]:
from syft.frameworks.torch.he.fv.modulus import CoeffModulus
from syft.frameworks.torch.he.fv.encryption_params import EncryptionParams
from syft.frameworks.torch.he.fv.context import Context
from syft.frameworks.torch.he.fv.integer_encoder import IntegerEncoder
from syft.frameworks.torch.he.fv.key_generator import KeyGenerator
from syft.frameworks.torch.he.fv.encrypter import Encrypter
from syft.frameworks.torch.he.fv.decrypter import Decrypter
from syft.frameworks.torch.he.fv.integer_encoder import IntegerEncoder
from syft.frameworks.torch.he.fv.modulus import SeqLevelType

## Generates coeff_modulus using Polymodulus degree and bit-length parameters.
Generates random integer of the given bit length for using as coeff_modulus

In [38]:
params = EncryptionParams()
cm = CoeffModulus()
params.coeff_modulus = cm.create(4096, [30, 50, 60])
print("coeff_modulus: ", params.coeff_modulus)

coeff_modulus:  [1073692673, 1125899906826241, 1152921504606830593]


## Use standard values of coeff_modulus for the given Poly Modulus degree and security_level.
Need some refractoring and integration of code.

In [39]:
params_standard = EncryptionParams()
cm_standard = CoeffModulus()
for i in SeqLevelType:
    for j in [1024, 4096, 32768]:           # ... [1024, 2048, 4096, 8192, 16384, 32768]
        params_standard.coeff_modulus = cm_standard.bfv_default(j, i)
        print(f"coeff_modulus for {i} and poly_modulus = {j}", params_standard.coeff_modulus)
    print('\n')

coeff_modulus for SeqLevelType.TC128 and poly_modulus = 1024 [132120577]
coeff_modulus for SeqLevelType.TC128 and poly_modulus = 4096 [68719403009, 68719230977, 137438822401]
coeff_modulus for SeqLevelType.TC128 and poly_modulus = 32768 [36028797017456641, 36028797014704129, 36028797014573057, 36028797014376449, 36028797013327873, 36028797013000193, 36028797012606977, 36028797010444289, 36028797009985537, 36028797005856769, 36028797005529089, 36028797005135873, 36028797003694081, 36028797003563009, 36028797001138177, 72057594037338113]


coeff_modulus for SeqLevelType.TC192 and poly_modulus = 1024 [520193]
coeff_modulus for SeqLevelType.TC192 and poly_modulus = 4096 [33538049, 33349633, 33292289]
coeff_modulus for SeqLevelType.TC192 and poly_modulus = 32768 [18014398506729473, 18014398505943041, 18014398499848193, 18014398498799617, 18014398498275329, 36028797017456641, 36028797014704129, 36028797014573057, 36028797014376449, 36028797013327873, 36028797013000193]


coeff_modulus for Se

## Keygeneration

In [40]:
params = EncryptionParams(128, CoeffModulus().create(128, [40, 40]), 1<<7 )
ctx = Context(params)
keygenerator = KeyGenerator(ctx)
sk, pk = keygenerator.keygen()

In [41]:
print('secret key values : ', sk.data)
print(sk)

secret key values :  [1, 0, 1099511607040, 0, 0, 1, 1, 0, 1, 1099511607040, 1099511607040, 1099511607040, 0, 1099511607040, 1, 1, 0, 0, 1, 0, 1, 1099511607040, 1099511607040, 1099511607040, 0, 1, 1099511607040, 1099511607040, 0, 0, 0, 1099511607040, 1099511607040, 0, 1099511607040, 0, 1, 1099511607040, 1099511607040, 1, 0, 0, 1099511607040, 1099511607040, 1, 1, 1099511607040, 0, 1, 1099511607040, 1099511607040, 1099511607040, 1099511607040, 0, 1099511607040, 1, 1, 1, 0, 0, 0, 0, 1099511607040, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1099511607040, 0, 1, 0, 1, 0, 1, 1, 1099511607040, 1, 1, 1099511607040, 1099511607040, 0, 1, 0, 1, 1099511607040, 1099511607040, 1099511607040, 1, 1, 1099511607040, 0, 0, 1, 0, 1, 1099511607040, 1099511607040, 0, 1, 0, 1099511607040, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1099511607040, 0, 1, 1, 0, 1099511607040, 0, 0, 1099511607040, 1099511607040, 1099511607040, 1, 0, 1099511619840, 0, 0, 1, 1, 0, 1, 1099511619840, 1099511619840, 1099511619840, 0, 1099511619840, 1, 1, 0, 0, 

In [42]:
print('public key values : ', pk.data)
print(pk)

public key values :  [[509693613655, 1099511607038, 144195502330, 2, 7, 674967841465, 310923196917, 4, 642983827178, 683781018082, 867393222460, 179507732797, 0, 318025229695, 967958001535, 1010047298927, 3, 1099511607040, 846044357090, 5, 34510438806, 634853024217, 967040677113, 1033218970405, 1099511607032, 63773619155, 506478838963, 601786231723, 1099511607038, 1099511607040, 1, 407490485866, 396370534583, 0, 521545732449, 3, 21378516475, 662788481271, 551906950056, 688541910800, 1099511607039, 0, 77084564845, 1019518191990, 125357708746, 640207367196, 670535480401, 1099511607040, 613142369006, 786518656264, 428657477858, 552297067750, 615424375017, 1099511607036, 1072256380720, 461454308239, 856624453456, 848373310925, 0, 0, 4, 0, 6149317100, 747687740568, 6, 1099511607034, 8296202603, 231407195065, 859407732403, 565612089610, 304026080392, 543669968817, 1099511607035, 516012375659, 1099511607038, 46957285294, 6, 807172278044, 2, 227211109333, 962550506107, 841984344245, 5187171067

## Integer Encoder
Encodes Integer values to Plaintext object

In [43]:
int_encoder = IntegerEncoder(ctx)
plaintext = int_encoder.encode(-20000)
print(plaintext)
# print('plaintext data',plaintext.data)

### Decodes back to Integer

In [44]:
print(int_encoder.decode(plaintext))

-20000


## Encrypter
Encrypt Plaintext to ciphertext using public_key

In [45]:
encrypter = Encrypter(ctx, pk)
ciphertext = encrypter.encrypt(plaintext)
print(ciphertext)
print('ciphertext data :', ciphertext.data)

ciphertext data : [[6, 5, 955316104714, 1099511607040, 1099511607030, 415953831149, 1099511607040, 4, 1099511607038, 407140654530, 223528450148, 170917798364, 0, 318025229695, 1090921672607, 1010047298926, 4, 2, 846044357088, 1099511607036, 1065001168235, 464658582823, 132470929929, 1099511607037, 2, 0, 506478838958, 1, 1, 1099511607039, 6, 692021121175, 396370534582, 0, 577965874592, 3, 1078133090559, 1099511607040, 1, 688541910800, 1099511607040, 1099511607040, 77084564846, 79993415050, 1099511607038, 640207367198, 670535480398, 1099511607037, 0, 0, 670854129184, 1099511607040, 1, 2, 1072256380717, 638057298802, 1099511607040, 1099511607040, 1099511607039, 1099511607036, 1, 3, 6149317104, 747687740569, 0, 5, 1091215404438, 868104411971, 240103874630, 533899517425, 795485526641, 0, 1099511607030, 583499231380, 4, 1052554321744, 1099511607039, 807172278046, 1099511607037, 227211109333, 1099511607038, 841984344247, 518717106729, 589217120252, 78258730684, 1076719265012, 0, 1099511607040

Encrypt Plaintext to ciphertext using secret_key

In [46]:
encrypter_symmetric = Encrypter(ctx, sk)
ciphertext_symmetric = encrypter_symmetric.encrypt(plaintext)
print(ciphertext_symmetric.data)

[[563890831488, 1099511607039, 1083994413754, 0, 1099511607038, 86888487808, 890879329558, 2, 557054797029, 245546226056, 329161413032, 30917407941, 3, 35093257026, 2171134692, 463020661978, 2, 1099511607040, 735122944281, 2, 658145587167, 997326750397, 725733719288, 545680370930, 1, 65196771949, 187709218323, 562172269306, 2, 7, 0, 949072085794, 495613361852, 1, 386501202582, 0, 888572615322, 229832110302, 1087237003894, 480785491549, 1099511607040, 1099511607035, 631798231977, 842109181589, 614067508785, 724314676670, 306039486348, 6, 433351775647, 515510109008, 119800199985, 710331560283, 1037853735556, 1099511607036, 774036941229, 477190176651, 367515486699, 169885861855, 1099511607039, 2, 1099511607040, 4, 634116736300, 624484865311, 1099511607040, 0, 249786101842, 7412081058, 408652104254, 99206112543, 616977021578, 387441810988, 1099511607039, 110075619961, 0, 400171659518, 1099511607039, 828411152280, 1, 562254515517, 934934204801, 655172392402, 663868608750, 913397089729, 2039

## Decrypter
Decrypt Ciphertext to Plaintext using secret_key

In [47]:
decrypter = Decrypter(ctx, sk)
result = decrypter.decrypt(ciphertext)
print("len : ",len(result.data))
print(result.data)
print(result)

len :  15
[0, 0, 0, 0, 0, 127, 0, 0, 0, 127, 127, 127, 0, 0, 127]


In [48]:
print(int_encoder.decode(result))

-20000
